### Import Required Libraries and Set Up Environment Variables

In [18]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [19]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
tmdb_api_token = os.getenv("TMDB_API_TOKEN")



### Access the New York Times API

In [20]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url= (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    +f"&fq={filter_query}&sort={sort}&qfl={field_list}"
)



In [21]:
# Create an empty list to store the reviews
articles_list = []

# loop through pages 0-19
for page in range(0, 5):

    # create query with a page number   
    # API results show 10 articles at a time

    query_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles["response"]["docs"]:

            articles_list.append(article)        

        # Print the page number that had no results then break from the loop
    except:
        print(f"You ended at page {page}")
        
        break

    # Print the page that was just retrieved
    print(f"Page {page} was successfully added")

Page 0 was successfully added
Page 1 was successfully added
Page 2 was successfully added
Page 3 was successfully added
Page 4 was successfully added


In [22]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(articles_list[:5], indent=4))

[
    {
        "abstract": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "lead_paragraph": "A trashy treat coated in a high-art gloss, \u201cThe Attachment Diaries\u201d gleefully kneads melodrama, noir, horror and sexual perversion into a pathological romance between two deeply damaged women.",
        "print_section": "C",
        "print_page": "7",
        "source": "The New York Times",
        "multimedia": [
            {
                "rank": 0,
                "subtype": "xlarge",
                "caption": null,
                "credit": null,
                "type": "image",
                "url": "images/2023/05/26/multimedia/attachment1-mbcw/attachment1-mbcw-articl

In [23]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
articles_list_df = pd.json_normalize(articles_list)
articles_list_df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,A film adaptation of Taylor Jenkins Reid’s nov...,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,"A woman stands on a dock, staring out at the s...",C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,There’s not much Lennon music heard in this do...,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,Interest in John Lennon’s personal life goes b...,C,6,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Gl

In [24]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

import re

def extracted_title(headline_main):
    extracted_title = re.search(r'\u2018(.*?)\u2019 Review', headline_main)
    if extracted_title:
        return extracted_title.group(1)
    else:
        return None

articles_list_df['title'] = articles_list_df['headline.main'].apply(lambda x: extracted_title(x))
articles_list_df.head(5)

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,None
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [25]:
# Extract 'name' and 'value' from items in "keywords" column

def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


# Fix the "keywords" column by converting cells from a list to a string
articles_list_df['keywords'] = articles_list_df['keywords'].apply(extract_keywords)
articles_list_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,None
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [26]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = articles_list_df['title'].to_list()
title_list

['The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',


### Access The Movie Database API

In [27]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
api_key_string = "&api_key=" + tmdb_api_key

In [29]:
# Create an empty list to store the results
tmdb_movie_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests

x=0

while x < 50: 
       
# Loop through the titles

    # Check if we need to sleep before making a request
    if x > 0 and x % 50 == 0:
            
            time.sleep(10)

    # Add 1 to the request counter

    x += 1
 
    # Perform a "GET" request for The Movie Database
   
    response = requests.get(f'{url}{title_list[x]}{api_key_string}').json()
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:

        # Get movie id
        movie_id = response["results"][0]["id"]
        

        # Make a request for a the full movie details
        details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?{api_key_string}'
    

    # Execute "GET" request with url
        
        full_details = requests.get(details_url + api_key_string).json()
         
            
    # Extract the genre names into a list
        
        genres = [full_details['genres'][i]['name'] for i in range(len(full_details['genres']))]



    # Extract the spoken_languages' English name into a list
            
        spoken_languages = [full_details['spoken_languages'][i]['english_name'] for i in range(len(full_details['spoken_languages']))]

        # Extract the production_countries' name into a list
        
        production_countries = [full_details['production_countries'][i]['name'] for i in range(len(full_details['production_countries']))]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            "title": full_details['title'],
            "original_title": full_details['original_title'],
            "budget": full_details['budget'],
            "original_language" : full_details['original_language'],
            "homepage": full_details['homepage'],
            "overview": full_details['overview'],
            "popularity": full_details['popularity'],
            "runtime": full_details['runtime'],
            "release_date": full_details['release_date'],
            "revenue": full_details['revenue'],
            "vote_average": full_details['vote_average'],
            "vote_count": full_details['vote_count'],
            "genre_names": full_details['genres'],
            "spoken_languages": full_details['spoken_languages'],
            "production_countries": full_details['production_countries'],
        }
        tmdb_movie_list.append(movie_data)
        
        # Print out the title that was found
        print(f'{full_details['original_title']} was found.')

    except Exception as e:
        print(f'Error: {e}')
        print("Movie not found")



None was found.
You Can Live Forever was found.
A Tourist's Guide to Love was found.
Les Enfants des autres was found.
One True Loves was found.
The Lost Weekend: A Love Story was found.
A Thousand and One was found.
Your Place or Mine was found.
Love in the Time of Fentanyl was found.
El apego was found.
None was found.
You Can Live Forever was found.
A Tourist's Guide to Love was found.
Les Enfants des autres was found.
One True Loves was found.
The Lost Weekend: A Love Story was found.
A Thousand and One was found.
Your Place or Mine was found.
Love in the Time of Fentanyl was found.
El apego was found.
None was found.
You Can Live Forever was found.
A Tourist's Guide to Love was found.
Les Enfants des autres was found.
One True Loves was found.
The Lost Weekend: A Love Story was found.
A Thousand and One was found.
Your Place or Mine was found.
Love in the Time of Fentanyl was found.
El apego was found.
None was found.
You Can Live Forever was found.
A Tourist's Guide to Love was f

IndexError: list index out of range

In [30]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movie_list[:5], indent=4))


[
    {
        "title": "Red Light Green Light",
        "original_title": "None",
        "budget": 0,
        "original_language": "en",
        "homepage": "",
        "overview": "Seven college students are forced to compete in children\u2019s games for survival. With their futures unclear, the secrets of their pasts are revealed.",
        "popularity": 0.6,
        "runtime": 75,
        "release_date": "",
        "revenue": 0,
        "vote_average": 0.0,
        "vote_count": 0,
        "genre_names": [
            {
                "id": 878,
                "name": "Science Fiction"
            },
            {
                "id": 27,
                "name": "Horror"
            }
        ],
        "spoken_languages": [
            {
                "english_name": "English",
                "iso_639_1": "en",
                "name": "English"
            }
        ],
        "production_countries": [
            {
                "iso_3166_1": "US",
                "nam

In [41]:
# Convert the results to a DataFrame
tmdb_movie_df = pd.DataFrame(tmdb_movie_list)
tmdb_movie_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,Red Light Green Light,None,0,en,,Seven college students are forced to compete i...,0.600,75,,0,0.000,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,2023-03-24,15055,6.700,33,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,2023-04-21,0,6.300,158,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,2022-09-21,84178,6.842,184,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.097,100,2023-04-07,37820,6.406,69,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."


### Merge and Clean the Data for Export

In [42]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(articles_list_df, tmdb_movie_df, on="title")   


In [53]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_fix = ["genre_names", "spoken_languages", "production_countries"]

# Create a list of characters to remove

chars_remove = ["[", "]", "'"]

# Loop through the list of columns to fix

for column in columns_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_remove:
        merged_df[column] = merged_df[column].apply(lambda x: x.replace(char, "") if isinstance(x, str) else x)

# Display the fixed DataFrame
merged_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.0,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
1,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.0,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
2,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.0,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
3,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.0,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
4,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,2023-03-24,15055,6.7,33,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CA, name: Canada}, {iso_3166_1: U..."


In [51]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)

KeyError: "['byline.person'] not found in axis"

In [55]:
# Delete duplicate rows and reset index
merged_df['multimedia'] = merged_df['multimedia'].astype(str)
merged_df.drop_duplicates(inplace=True)
merged_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.000,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
4,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,2023-03-24,15055,6.700,33,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CA, name: Canada}, {iso_3166_1: U..."
9,A film adaptation of Taylor Jenkins Reid’s nov...,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,"A woman stands on a dock, staring out at the s...",C,8,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...","subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,...,Emma and Jesse are living the perfect life tog...,14.097,100,2023-04-07,37820,6.406,69,"{id: 10749, name: Romance}, {id: 18, name: Dra...","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CZ, name: Czech Republic}, {iso_3..."
14,There’s not much Lennon music heard in this do...,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,Interest in John Lennon’s personal life goes b...,C,6,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,...,May Pang lovingly recounts her life in rock & ...,7.393,95,2023-04-13,77145,6.000,2,"{id: 99, name: Documentary}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"
19,A mesmerizing Teyana Taylor stars in A.V. Rock...,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,"The first time you truly see Inez De La Paz, t...",C,1,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,...,Struggling but unapologetically living on her ...,13.351,116,2023-03-31,3395595,6.905,84,"{id: 18, name: Drama}, {id: 80, name: Crime}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"


In [56]:
# Export data to CSV without the index
merged_df.to_csv("output/NYT_TMDB.csv", index=False)
merged_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,2021-10-07,0,3.000,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
4,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,2023-03-24,15055,6.700,33,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CA, name: Canada}, {iso_3166_1: U..."
9,A film adaptation of Taylor Jenkins Reid’s nov...,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,"A woman stands on a dock, staring out at the s...",C,8,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...","subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,...,Emma and Jesse are living the perfect life tog...,14.097,100,2023-04-07,37820,6.406,69,"{id: 10749, name: Romance}, {id: 18, name: Dra...","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CZ, name: Czech Republic}, {iso_3..."
14,There’s not much Lennon music heard in this do...,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,Interest in John Lennon’s personal life goes b...,C,6,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,...,May Pang lovingly recounts her life in rock & ...,7.393,95,2023-04-13,77145,6.000,2,"{id: 99, name: Documentary}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"
19,A mesmerizing Teyana Taylor stars in A.V. Rock...,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,"The first time you truly see Inez De La Paz, t...",C,1,The New York Times,"0 [{'rank': 0, 'subtype': 'xlarge', 'capt...",subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,...,Struggling but unapologetically living on her ...,13.351,116,2023-03-31,3395595,6.905,84,"{id: 18, name: Drama}, {id: 80, name: Crime}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"
